# Feature selection

In [1]:
import os
import time
from math import log

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import Lasso, LassoCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from IPython.display import display
pd.options.display.max_rows = 80

t = time.time()

In [2]:
df = pd.read_csv('data/df_train_prepro.csv').sample(frac=1)
df.shape

(1610494, 48)

In [3]:
columns_quant = ['contextid',
 'campaignctrlast24h',
 'dayssincelastvisitdouble',
 'ltf_nbglobaldisplay_4w',
 'ltf_nbpartnerdisplayssincelastclick',
 'ltf_nbpartnerdisplay_90d',
 'ltf_nbpartnerclick_90d',
 'ltf_nbpartnersales_90d',
 'nbdayssincelastclick',
 'nbdisplay_1hour',
 'nbdisplayglobalapprox_1d_sum_xdevice',
 'display_size',
 'zonecostineuro']

columns_cat = ['display_env',
 'target_env',
 'campaignscenario',
 'campaignvertical',
 'is_interstitial',
 'device_type',
 'hour',
 'weekday']

X_quant = df[columns_quant]
X_quant_scaled = StandardScaler().fit_transform(X_quant)
print(len(X_quant.columns))
display(X_quant.columns)

X_cat = df[columns_cat]
X_cat = pd.get_dummies(X_cat, columns=columns_cat, drop_first=True)
X_cat_scaled = StandardScaler().fit_transform(X_cat)
print(len(X_cat.columns))
display(X_cat.columns)

X_quant_cat = df[columns_quant + columns_cat]
X_quant_cat = pd.get_dummies(X_quant_cat, columns=columns_cat, drop_first=True)
X_quant_cat_scaled = StandardScaler().fit_transform(X_quant_cat)
print(len(X_quant_cat.columns))
display(X_quant_cat.columns)

y = df['is_display_clicked']

13


Index(['contextid', 'campaignctrlast24h', 'dayssincelastvisitdouble',
       'ltf_nbglobaldisplay_4w', 'ltf_nbpartnerdisplayssincelastclick',
       'ltf_nbpartnerdisplay_90d', 'ltf_nbpartnerclick_90d',
       'ltf_nbpartnersales_90d', 'nbdayssincelastclick', 'nbdisplay_1hour',
       'nbdisplayglobalapprox_1d_sum_xdevice', 'display_size',
       'zonecostineuro'],
      dtype='object')

50


Index(['display_env_app_ios', 'display_env_other', 'display_env_web',
       'target_env_2', 'campaignscenario_5', 'campaignscenario_6',
       'campaignscenario_11', 'campaignscenario_12', 'campaignscenario_13',
       'campaignscenario_17', 'campaignvertical_20.0', 'campaignvertical_23.0',
       'campaignvertical_129.0', 'campaignvertical_200002.0',
       'is_interstitial_True', 'device_type_Desktop',
       'device_type_Desktop - Safari', 'device_type_Mobile - Other',
       'device_type_Unknown', 'device_type_iPad', 'device_type_iPhone',
       'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5', 'hour_6', 'hour_7',
       'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12', 'hour_13',
       'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18', 'hour_19',
       'hour_20', 'hour_21', 'hour_22', 'hour_23', 'weekday_Monday',
       'weekday_Saturday', 'weekday_Sunday', 'weekday_Thursday',
       'weekday_Tuesday', 'weekday_Wednesday'],
      dtype='object')

63


Index(['contextid', 'campaignctrlast24h', 'dayssincelastvisitdouble',
       'ltf_nbglobaldisplay_4w', 'ltf_nbpartnerdisplayssincelastclick',
       'ltf_nbpartnerdisplay_90d', 'ltf_nbpartnerclick_90d',
       'ltf_nbpartnersales_90d', 'nbdayssincelastclick', 'nbdisplay_1hour',
       'nbdisplayglobalapprox_1d_sum_xdevice', 'display_size',
       'zonecostineuro', 'display_env_app_ios', 'display_env_other',
       'display_env_web', 'target_env_2', 'campaignscenario_5',
       'campaignscenario_6', 'campaignscenario_11', 'campaignscenario_12',
       'campaignscenario_13', 'campaignscenario_17', 'campaignvertical_20.0',
       'campaignvertical_23.0', 'campaignvertical_129.0',
       'campaignvertical_200002.0', 'is_interstitial_True',
       'device_type_Desktop', 'device_type_Desktop - Safari',
       'device_type_Mobile - Other', 'device_type_Unknown', 'device_type_iPad',
       'device_type_iPhone', 'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5',
       'hour_6', 'hour_7', 'hour_

## Méthode LASSO

On choisit `fit_intercept=False` et `normalize=False` car les données sont déjà centrées et réduites.

In [4]:
def run_LassoCV(alphas_range, X, y, X_columns=False):
    lcv = LassoCV(alphas=alphas_range, normalize=False, fit_intercept=False, random_state=0, n_jobs=-1).fit(X, y)

    print(f"Best alpha : {lcv.alpha_}")

    lasso = Lasso(fit_intercept=False, normalize=False, alpha=lcv.alpha_)
    lasso.fit(X, y)
    
    if type(X_columns) == bool:
        X_columns = X.columns
    
    coeff_df = pd.DataFrame(np.transpose(lasso.coef_), X_columns, columns=['Coefficient abs']).abs()

    display(coeff_df.sort_values(by='Coefficient abs', ascending=False))

    print(f"\nNombre de coeffs non nuls : {len(coeff_df[coeff_df['Coefficient abs'] != 0])}")

### Variables quantitatives

In [5]:
%%time
alphas_range = np.linspace(0.001, 1, num=101)
print(alphas_range[:4], '...', alphas_range[-3:])
run_LassoCV(alphas_range, X_quant_scaled, y, X_columns=X_quant.columns)

[0.001   0.01099 0.02098 0.03097] ... [0.98002 0.99001 1.     ]
Best alpha : 0.001


,Coefficient abs
zonecostineuro,0.043662
display_size,0.023909
campaignctrlast24h,0.023156
nbdisplay_1hour,0.007687
nbdayssincelastclick,0.003271
contextid,0.003043
ltf_nbpartnerclick_90d,0.002896
nbdisplayglobalapprox_1d_sum_xdevice,0.002280
ltf_nbpartnerdisplayssincelastclick,0.000031
dayssincelastvisitdouble,0.000000



Nombre de coeffs non nuls : 9
Wall time: 4.04 s


Le best alpha correspond à la borne inférieure de l'intervalle, nous continuons donc la recherche du best alpha.

In [6]:
%%time
alphas_range = np.linspace(0.000001, 0.002, num=101)
print(alphas_range[:4], '...', alphas_range[-3:])
run_LassoCV(alphas_range, X_quant_scaled, y, X_columns=X_quant.columns)

[1.000e-06 2.099e-05 4.098e-05 6.097e-05] ... [0.00196002 0.00198001 0.002     ]
Best alpha : 1e-06


,Coefficient abs
zonecostineuro,0.044570
display_size,0.024767
campaignctrlast24h,0.024182
nbdisplay_1hour,0.008354
ltf_nbpartnerclick_90d,0.004820
nbdayssincelastclick,0.004108
contextid,0.004080
nbdisplayglobalapprox_1d_sum_xdevice,0.003093
ltf_nbpartnerdisplay_90d,0.001534
ltf_nbpartnersales_90d,0.000791



Nombre de coeffs non nuls : 13
Wall time: 5.55 s


In [7]:
%%time
alphas_range = np.linspace(0.0000001, 0.00005, num=101)
print(alphas_range[:4], '...', alphas_range[-3:])
run_LassoCV(alphas_range, X_quant_scaled, y, X_columns=X_quant.columns)

[1.000e-07 5.990e-07 1.098e-06 1.597e-06] ... [4.9002e-05 4.9501e-05 5.0000e-05]
Best alpha : 9.082e-06


,Coefficient abs
zonecostineuro,0.044562
display_size,0.024760
campaignctrlast24h,0.024173
nbdisplay_1hour,0.008350
ltf_nbpartnerclick_90d,0.004801
nbdayssincelastclick,0.004100
contextid,0.004072
nbdisplayglobalapprox_1d_sum_xdevice,0.003086
ltf_nbpartnerdisplay_90d,0.001507
ltf_nbpartnersales_90d,0.000780



Nombre de coeffs non nuls : 13
Wall time: 5.05 s


Nous pouvons en conclure que la variable `dayssincelastvisitdouble` n'est pas utile.

### Variables quantitatives + catégorielles
On rajoute à présent les variables catégorielles.

In [8]:
%%time
alphas_range = np.linspace(0.001, 1, num=101)
print(alphas_range[:4], '...', alphas_range[-3:])
run_LassoCV(alphas_range, X_quant_cat_scaled, y, X_columns=X_quant_cat.columns)

[0.001   0.01099 0.02098 0.03097] ... [0.98002 0.99001 1.     ]
Best alpha : 0.001


,Coefficient abs
is_interstitial_True,0.039362
zonecostineuro,0.036240
campaignctrlast24h,0.016760
device_type_Desktop,0.007656
display_size,0.007503
nbdisplay_1hour,0.004086
nbdayssincelastclick,0.003941
campaignscenario_13,0.003517
device_type_iPhone,0.002767
ltf_nbpartnerclick_90d,0.002765



Nombre de coeffs non nuls : 25
Wall time: 16.1 s


In [9]:
%%time
alphas_range = np.linspace(0.000001, 0.002, num=101)
print(alphas_range[:4], '...', alphas_range[-3:])
run_LassoCV(alphas_range, X_quant_cat_scaled, y, X_columns=X_quant_cat.columns)

[1.000e-06 2.099e-05 4.098e-05 6.097e-05] ... [0.00196002 0.00198001 0.002     ]
Best alpha : 2.099e-05


,Coefficient abs
is_interstitial_True,0.039178
zonecostineuro,0.037087
campaignctrlast24h,0.017210
device_type_Desktop,0.009434
display_size,0.008212
device_type_iPhone,0.006831
display_env_app_ios,0.006096
target_env_2,0.005785
campaignscenario_13,0.004896
nbdayssincelastclick,0.004873



Nombre de coeffs non nuls : 61
Wall time: 24.4 s


In [10]:
%%time
alphas_range = np.linspace(0.000001, 0.00005, num=100)
print(alphas_range[:4], '...', alphas_range[-3:])
run_LassoCV(alphas_range, X_quant_cat_scaled, y, X_columns=X_quant_cat.columns)

[1.00000000e-06 1.49494949e-06 1.98989899e-06 2.48484848e-06] ... [4.90101010e-05 4.95050505e-05 5.00000000e-05]
Best alpha : 2.0303030303030307e-05


,Coefficient abs
is_interstitial_True,0.039178
zonecostineuro,0.037087
campaignctrlast24h,0.017210
device_type_Desktop,0.009435
display_size,0.008213
device_type_iPhone,0.006834
display_env_app_ios,0.006100
target_env_2,0.005788
campaignscenario_13,0.004897
nbdayssincelastclick,0.004874



Nombre de coeffs non nuls : 61
Wall time: 24.2 s


## Variance feature selection

In [11]:
def low_variance_feature_selection(X, threshold) :
    sel = VarianceThreshold(threshold)
    sel.fit(X)
    print(sel.get_support(indices=True))

    df = pd.DataFrame(sel.variances_, index=X.columns, columns=['Variance']).sort_values(by='Variance', ascending=False)

    display(df)

    return sel.transform(X)

In [12]:
%%time
low_variance_feature_selection(X_quant, 0.01)

[ 0  2  3  4  5  6  7  8  9 10 11 12]


,Variance
display_size,9.444482e+09
ltf_nbglobaldisplay_4w,2.607098e+04
ltf_nbpartnerdisplay_90d,1.061908e+04
nbdisplayglobalapprox_1d_sum_xdevice,4.449996e+03
dayssincelastvisitdouble,2.642414e+03
nbdayssincelastclick,2.066988e+03
ltf_nbpartnerdisplayssincelastclick,7.590539e+02
nbdisplay_1hour,7.176346e+01
zonecostineuro,8.801236e+00
ltf_nbpartnerclick_90d,6.238911e+00


Wall time: 388 ms


array([[ 9.00000000e+00,  4.55000000e-01,  0.00000000e+00, ...,
         6.35575000e+01,  6.55200000e+04,  2.54259855e-01],
       [ 1.00000000e+00, -1.00000000e+00,  0.00000000e+00, ...,
         2.12822222e+01,  2.01600000e+04,  8.13000023e-01],
       [ 6.00000000e+00,  8.48730000e+01,  0.00000000e+00, ...,
         4.47180556e+01,  2.74400000e+05,  1.19999997e-01],
       ...,
       [ 6.00000000e+00,  2.90964000e+02,  0.00000000e+00, ...,
         3.50000000e+01,  0.00000000e+00,  1.02581635e-01],
       [ 9.00000000e+00,  9.49000000e-01,  0.00000000e+00, ...,
         7.13824074e+01,  1.80000000e+05,  6.69000030e-01],
       [ 7.00000000e+00,  5.68700000e+00,  0.00000000e+00, ...,
         3.30000000e+01,  0.00000000e+00,  1.33356035e+00]])

In [13]:
%%time
low_variance_feature_selection(X_cat, 0.01)

[ 0  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 18 19 20 24 25 26 27 28
 29 30 31 32 33 34 35 36 37 38 39 40 41 42 44 45 46 47 48 49]


,Variance
campaignscenario_13,0.232464
device_type_Desktop,0.216539
campaignvertical_20.0,0.211358
display_env_web,0.207430
weekday_Wednesday,0.129508
weekday_Sunday,0.128571
weekday_Thursday,0.123458
weekday_Tuesday,0.122092
weekday_Monday,0.121234
weekday_Saturday,0.115971


Wall time: 952 ms


array([[0, 1, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]], dtype=uint8)

## Univariate feature selection

In [14]:
from scipy.stats.stats import pearsonr
def univariate_selection(X, y) :
    features = list(X)
    correlation = []
    significance = []
    for feature in features:
        correl = pearsonr(X[feature].values, y.values)
        correlation.append(correl[0])
        significance.append(correl[1])
    df = pd.DataFrame()
    df['feature'] = features
    df['correlation'] = correlation
    df['abs_correlation'] = np.abs(correlation)
    df['significance'] = significance
    df['significant'] = df['significance'] < 0.01 # Label those P<0.01
    df.sort_values(by='abs_correlation', ascending=False, inplace=True)
    return df

In [15]:
%%time
univariate_selection(X_quant, y)

Wall time: 500 ms


,feature,correlation,abs_correlation,significance,significant
12,zonecostineuro,0.214615,0.214615,0.000000e+00,True
11,display_size,0.145901,0.145901,0.000000e+00,True
1,campaignctrlast24h,0.104729,0.104729,0.000000e+00,True
9,nbdisplay_1hour,-0.045473,0.045473,0.000000e+00,True
10,nbdisplayglobalapprox_1d_sum_xdevice,-0.032693,0.032693,0.000000e+00,True
8,nbdayssincelastclick,-0.024608,0.024608,3.742619e-214,True
6,ltf_nbpartnerclick_90d,0.023766,0.023766,6.851296e-200,True
2,dayssincelastvisitdouble,-0.022727,0.022727,5.873928e-183,True
0,contextid,0.021707,0.021707,4.346806e-167,True
5,ltf_nbpartnerdisplay_90d,0.004920,0.004920,4.269942e-10,True


In [16]:
%%time
univariate_selection(X_cat, y)

Wall time: 1.76 s


,feature,correlation,abs_correlation,significance,significant
14,is_interstitial_True,0.246777,0.246777,0.000000e+00,True
2,display_env_web,-0.087985,0.087985,0.000000e+00,True
3,target_env_2,0.067383,0.067383,0.000000e+00,True
15,device_type_Desktop,-0.059810,0.059810,0.000000e+00,True
0,display_env_app_ios,0.056112,0.056112,0.000000e+00,True
8,campaignscenario_13,-0.048137,0.048137,0.000000e+00,True
9,campaignscenario_17,0.040227,0.040227,0.000000e+00,True
19,device_type_iPad,0.034953,0.034953,0.000000e+00,True
20,device_type_iPhone,0.023572,0.023572,1.145756e-196,True
7,campaignscenario_12,-0.018978,0.018978,3.478621e-128,True


In [17]:
print(f"Temps d'exécution total : {time.strftime('%H:%M:%S', time.gmtime(time.time()-t))}")

Temps d'exécution total : 00:01:45
